In [ ]:
import numpy as np
import plotly.express as px

def calc_p_impact(x,y,original_swap_x,fee):
    """
    @notice calculate slippage when selling x or y to the open market.
    Assuming that we trade against Uniswap is a conservative assumption
    """
    # TODO: Also need to incorporate original_swap_y in case we are selling collateral
    # TODO: Need to incorporate concentrated liquidity (e.g., no more liquidity beyond ]a, b[)
    # TODO: Need to incorproate the StableSwap invariant for crvUSD pool liquidity
    # x = 2e6 
    # y = 1e3
    k = x*y
    original_price = x/y
    # original_swap_x = 10e3
    # fee=0.00
    swap_x = original_swap_x*(1-fee)
    new_x = x + swap_x
    new_y = k/new_x
    swap_y = y-new_y
    trade_price = swap_x/swap_y
    new_price = new_x/new_y
    return(trade_price)

calc_p_impact(2e6,1e3,10e3,0.00) 
x = np.linspace(1,2e6, num=500)
y = [calc_p_impact(2e6,1e3,x_i,0.00) for x_i in x]
fig = px.scatter(x=x, y=y)
fig.show()

In [ ]:
# Empirical slippage analysis
import csv
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

with open("../data/weth_usdc_005_tier_trades.csv","r") as infile:
    data = pd.read_csv(infile)

with open("../data/weth_usdc_005_tier_trades2.csv","r") as infile:
    data2 = pd.read_csv(infile)

data3 = pd.concat([data,data2]).drop_duplicates()
data3["WETH"]=data3["amount1"].apply(lambda x: int(x) / 1e18)
data3["USDC"]=data3["amount0"].apply(lambda x: int(x)/1e6)
data3["price_implied"] = data3["price_implied"]/1e18
data3["price_actual"] = data3["price_actual"]/1e18
data3["price_impact"]=abs(data3["price_implied"]-data3["price_actual"])/data3["price_implied"]
data3["evt_block_time"] = pd.to_datetime(data3['evt_block_time'], utc=True)
data3['unix_timestamp'] = data3['evt_block_time'].astype(int) // 10**9
data3.sort_values(by="unix_timestamp", axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
data3["volatility"]=data3["price_actual"].rolling(30).std()

data3 = data3.dropna(subset=['volatility'])
data3['volatility_bin'] = (data3['volatility']*100).astype("int")/100
# data3['WETH_bin'] = np.power(10, np.log10(data3['WETH']).round())
data3['WETH_bin'] = data3['WETH'].round()
data4=data3.groupby(["volatility_bin","WETH_bin"]).agg({"price_impact": "mean"}).reset_index()
print(data4[["volatility_bin","WETH_bin","price_impact"]])
fig = px.scatter_3d(data4,x="volatility_bin",y="WETH_bin",z="price_impact",color="price_impact")
fig.show()

# Linear regression
# Generate Y-values based on the regression model
slope, intercept = np.polyfit(data3["WETH"], data3["price_impact"], 1)
y_fit = slope * data3["WETH"] + intercept
print(slope,intercept)
plt.scatter(x=data3["WETH"], y=data3["price_impact"], label='Data', color='blue')
plt.plot(data3["WETH"], y_fit, label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', color='red')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()



In [ ]:
# new slippage function based on empirical analysis
def quad_output(x):
    """
    @notice calculate price impact when selling x or y to the open market.
    Assuming that we trade against Uniswap is a conservative assumption
    """
    a,b,c = (2.602424822058012e-10,2.3009044333523045e-07,0.0004613261845851614)
    return(a*x**2+b*x+c)

x = np.logspace(-9,5, num=50, endpoint=True, base=10.0, dtype=None, axis=0)
y = [quad_output(x) for x in x]
fig=px.line(x=x, y=y)
fig.show()

def lin_output(x):
    return 1.081593506690093e-06*x+0.0004379110082802476

x = np.logspace(-9,5, num=50, endpoint=True, base=10.0, dtype=None, axis=0)
print(x)
y = [lin_output(x) for x in x]
fig=px.line(x=x, y=y)
fig.show()

In [ ]:
import pandas as pd
with open("../data/weth_usdc_005_tier_trades.csv","r") as infile:
    data = pd.read_csv(infile)

with open("../data/weth_usdc_005_tier_trades2.csv","r") as infile:
    data2 = pd.read_csv(infile)

data3 = pd.concat([data,data2]).drop_duplicates()
data3 = data3[["amount0","amount1","price_implied","price_actual","previous_price_actual","price_movement_percentage","evt_block_time"]]

print(data3.shape)
